In [98]:
# import modules
import pandas as pd
import numpy as np
import spotipy

# jupyter notebook full-width display
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# pandas formatting
pd.set_option('display.float_format', '{:_.0f}'.format)
# NOTE: underscore separaters ('_') are better than commas (',') because 
# numbers with underscores work in Python without any extra effort.
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 200)

### The Billboard 100
https://www.kaggle.com/datasets/dhruvildave/billboard-the-hot-100-songs

In [2]:
# Billboard Top 100 Historical Data
# via:  https://toolbox.google.com/datasetsearch
url_billboard = r'D:\RYERSON\820\Datasets\Billboard The Hot 100 Songs\charts.csv'

df_billboard = pd.read_csv(url_billboard)
df_billboard['date'] = pd.to_datetime(df_billboard['date'])

# Unique Songs from The Billboard 100 Dataset

# just the songs on the billboard 100, once per song
df_billboard_songs = df_billboard[['song', 'artist']].drop_duplicates().sort_values(['artist', 'song']).reset_index(drop=True)

# add a blank id column and a blank MISSING column 
df_billboard_songs['id'] = ''
df_billboard_songs['MISSING'] = ''

df_billboard.shape, df_billboard_songs.shape

((330087, 7), (29681, 4))

### 1.2M Songs with Metadata (csv)
https://www.kaggle.com/datasets/rodolfofigueroa/spotify-12m-songs

In [3]:
# Spotify 1.2M+ Songs
# via:  https://www.kaggle.com/datasets/
url_1M_songs = r'D:\RYERSON\820\Datasets\Spotify 1.2M+ Songs\tracks_features.csv'

# create the dataframe with the large number of songs metadata
df_1M_songs = pd.read_csv(url_1M_songs)

# make a list of song ids from the 1M dataset
metadata_ids_csv = df_1M_songs.id.to_list()

### 8.7M Songs with Metadata (SQL)

In [4]:
# all ids from the SQLite database
metadata_ids_SQL = pd.read_csv('all_ids_sql.csv', header=None, names=['id'])
metadata_ids_SQL = metadata_ids_SQL.id.to_list()

### Combine Ids For Datasets with Metadata

In [5]:
all_metadata_ids = set(metadata_ids_csv + metadata_ids_SQL) # set() faster to search, and no duplicates
len(all_metadata_ids)

9592981

### Get Track IDs using API

### TEMPORARY TOKEN WORKFLOW

##### get a temporary authorization token from: https://developer.spotify.com/console/get-search-item

In [6]:
# input the temporary token
TEMP_TOKEN = input('Enter token: ')

# create a spotify object
spotify = spotipy.Spotify(auth=TEMP_TOKEN)

Enter token: BQDmmPusHl5G46Nv_X_jhw6bspvKSoFDn1UySHQxIeRDx37yp6PwDhgZ3GCz37wMdyfe4BFvYmFynTGjdgDzvB8EbP5M99tm_ZOBdfFgn4UIAyGRQYwTYhKrpi9Db0aX-keyb3XCZI53l3b12DgnN2iwlEUG_VYD_cMNPTGr2Scw


In [62]:
## TODO, add a 'MISSING' tag somewhere

# helper function
def find_id(track_title, artist_name, metadata_ids):
    """ 
    for searches with multiple results, all id were identical for the test cases I ran 
    some searches return no results, in this case the song is not on spotify
        confirmed by spot checks in the spotify music player
    some tracks give a 404 error, 
        these seem to exist in Spotify but 404 anyway
        not sure why the API does this but 
    """
    track_info = spotify.search(q='artist:' + artist_name + ' track:' + track_title, type='track')
    
    if track_info['tracks']['items'] == []:  # if track doesn't exist on Spotify
        return '', 'MISSING'
    else:
        # default to 0th id
        track_id = track_info['tracks']['items'][0]['id']
        
        number_of_results = len(track_info['tracks']['items'])
        
        # check if there is a better match
        for i in range(number_of_results):
            current_id = track_info['tracks']['items'][i]['id']
            if current_id in metadata_ids:
                return current_id, 'matched'  # immediately return it if it's found
        
        # if we made it through the loop without returning, note 'MISSING' and return the 0th id
        return track_id, 'MISSING'

In [63]:
%%time
# TEST
find_id("You Can't Turn Me Off (In The Middle Of Turning Me On)", 'High Inergy', all_metadata_ids)

Wall time: 304 ms


('7CfCUjAZFndcpAWLogl9J3', 'matched')

In [65]:
# TEST
find_id("You Can't Find this (SONG)", 'Low Unergy', all_metadata_ids)

('', 'MISSING')

### Add Spotify IDs to billboard songs matched in the datasets

In [9]:
# load saved csv if required
df_billboard_songs = pd.read_csv('df_billboard_songs.csv', keep_default_na=False)
df_billboard_songs.id.nunique()

14291

In [10]:
# start over at
start_over_at = 30000

# populate df_billboard_songs with ids, where available
for i, row in df_billboard_songs.iterrows():
    
    # start over at
    if i < start_over_at-1:
        continue
    
    # show status update
    if i%10 == 0:
        print(i, end='  ')
    if i%100 == 0:
        print()
        
    # start over where we finished (don't overwrite known ids)
    if df_billboard_songs['id'].iloc[i] != '':  
        continue
    # append id, NONE, or 'ERROR'
    else:
        artist = row[1]
        song = row[0]
        try:
            # unless there is an error, will append the id or None 
            df_billboard_songs['id'].iloc[i], df_billboard_songs['MISSING'].iloc[i] = find_id(song, artist, all_metadata_ids)
        except:  # all errors treated the same
            # if there is an error, change id to 'ERROR'
            print('ERROR:  ', artist, song)
            df_billboard_songs['MISSING'].iloc[i] = 'ERROR'  # leave id blank
        
        # save every 1000 rows, if new
        if i%1000 == 0:
            df_billboard_songs.to_csv('df_billboard_songs_TEMP.csv', index=False)
        
# save final dataframe
df_billboard_songs.to_csv('df_billboard_songs.csv', index=False)

19060  19070  19080  19090  19100  


HTTP Error for GET to https://api.spotify.com/v1/search with Params: {'q': 'artist:P. Diddy & Ginuwine Featuring Loon, Mario Winans & Tammy Ruggeri track:I Need A Girl (Part Two)', 'limit': 10, 'offset': 0, 'type': 'track', 'market': None} returned 404 due to Not found.


ERROR:   P. Diddy & Ginuwine Featuring Loon, Mario Winans & Tammy Ruggeri I Need A Girl (Part Two)
19110  19120  19130  19140  19150  19160  19170  19180  19190  19200  
19210  19220  19230  19240  19250  19260  19270  19280  19290  19300  
19310  19320  19330  19340  19350  19360  19370  19380  19390  19400  
19410  19420  19430  19440  19450  19460  19470  19480  19490  19500  
19510  19520  19530  19540  19550  19560  19570  19580  19590  19600  
19610  19620  19630  19640  19650  19660  19670  

HTTP Error for GET to https://api.spotify.com/v1/search with Params: {'q': "artist:Perry Como And The Fontane Sisters With Mitchell Ayres And His Orchestra track:It's Beginning To Look A Lot Like Christmas", 'limit': 10, 'offset': 0, 'type': 'track', 'market': None} returned 404 due to Not found.


ERROR:   Perry Como And The Fontane Sisters With Mitchell Ayres And His Orchestra It's Beginning To Look A Lot Like Christmas
19680  19690  19700  
19710  19720  19730  19740  19750  19760  19770  19780  19790  19800  
19810  19820  19830  19840  19850  19860  19870  19880  19890  19900  
19910  

...

ERROR:   Thomas Rhett Featuring Reba McEntire, Hillary Scott, Chris Tomlin & Keith Urban Be A Light
27580  27590  27600  
27610  27620  27630  27640  27650  27660  27670  27680  27690  27700  
27710  27720  27730  27740  27750  27760  27770  27780  27790  27800  
27810  27820  27830  27840  27850  27860  27870  27880  27890  27900  
27910  27920  27930  27940  27950  27960  27970  27980  27990  28000  
28010  28020  28030  28040  28050  28060  28070  28080  28090  28100  
28110  28120  28130  28140  28150  28160  28170  28180  28190  28200  
28210  28220  28230  28240  28250  28260  28270  28280  28290  28300  
28310  28320  28330  28340  28350  28360  28370  28380  28390  28400  
28410  28420  28430  28440  28450  28460  28470  28480  

HTTP Error for GET to https://api.spotify.com/v1/search with Params: {'q': 'artist:Tyler, The Creator Featuring Brent Faiyaz & Fana Hues track:Sweet / I Thought You Wanted To Dance', 'limit': 10, 'offset': 0, 'type': 'track', 'market': None} returned 404 due to Not found.


ERROR:   Tyler, The Creator Featuring Brent Faiyaz & Fana Hues Sweet / I Thought You Wanted To Dance
28490  28500  
28510  28520  28530  28540  28550  28560  28570  28580  28590  28600  
28610  28620  28630  28640  28650  28660  28670  28680  28690  28700  
28710  28720  28730  28740  28750  28760  28770  28780  28790  28800  
28810  28820  28830  28840  28850  28860  28870  28880  28890  28900  
28910  28920  28930  28940  28950  28960  28970  28980  28990  29000  
29010  29020  29030  29040  29050  29060  29070  29080  29090  29100  
29110  29120  29130  29140  29150  29160  29170  29180  29190  29200  
29210  29220  29230  29240  29250  29260  29270  29280  29290  29300  
29310  29320  29330  29340  29350  29360  29370  29380  29390  29400  
29410  29420  29430  29440  29450  29460  29470  29480  29490  29500  
29510  29520  29530  29540  29550  29560  29570  29580  29590  29600  
29610  29620  29630  29640  29650  29660  29670  

HTTP Error for GET to https://api.spotify.com/v1/search with Params: {'q': "artist:will.i.am Featuring Miley Cyrus, French Montana, Wiz Khalifa & DJ Mustard track:Feelin' Myself", 'limit': 10, 'offset': 0, 'type': 'track', 'market': None} returned 404 due to Not found.


29680  ERROR:   will.i.am Featuring Miley Cyrus, French Montana, Wiz Khalifa & DJ Mustard Feelin' Myself


### After Gathering all IDs

In [137]:
df_billboard_songs = pd.read_csv('df_billboard_songs.csv', keep_default_na=False)

In [139]:
# how many id have we added
df_billboard_songs.id.nunique(), sum(df_billboard_songs.id != "")
# 83 duplicated ids

(22274, 22357)

### Remove Duplicates / Errors

In [140]:
duplicates = df_billboard_songs[df_billboard_songs.id!='']
duplicates = duplicates[duplicates.id.duplicated(False)]
duplicates.to_csv('duplicated_ids.csv', index=True)

In [100]:
# # go through this manually and fix it
#     # in this case NEED TO FIX BOTH BILLBOARD DATAFRAMES

# # OR drop the corrupted rows
# # manually imputing is problematic, error prone, and time consuming

# duplicates.sort_values('id').head(200)

In [141]:
# set of duplicated ids
duplicated_ids = set(duplicates.id)

In [142]:
sum(df_billboard_songs['id'].isin(duplicated_ids))

168

In [143]:
# drop from billboard list of known ids 
for i, row in df_billboard_songs.iterrows():
    if df_billboard_songs.iloc[i]['id'] in duplicated_ids:
        df_billboard_songs['MISSING'].iloc[i] = 'DUPLICATED'
        df_billboard_songs['id'].iloc[i] = ''

In [144]:
sum(df_billboard_songs['id'].isin(duplicated_ids))

0

In [145]:
df_billboard_songs.to_csv('df_billboard_songs - duplicates removed.csv', index=False)

### what songs are still missing audio feature data?

In [146]:
df_billboard_songs = pd.read_csv('df_billboard_songs - duplicates removed.csv', keep_default_na=False)

In [147]:
# how many id have we added
df_billboard_songs.id.nunique(), sum(df_billboard_songs.id != '')
# off by one because '' counts as a unique id

(22190, 22189)

In [148]:
# check how many id match the metadata_ids
sum(df_billboard_songs.id.isin(all_metadata_ids))

18616

In [152]:
# songs on spotify that we don't have audio features for yet
need_audio_features = df_billboard_songs[(~df_billboard_songs.id.isin(all_metadata_ids) & (df_billboard_songs.id != ''))]
need_audio_features.to_

In [153]:
len(need_audio_features.id)

3573

### Final Datasets

In [ ]:
# separate songs into
    # not on spotify (no id)
    # on spotify, but not either dataset (not in either metadata set)
    # matched with dataset (could distinguish using temp_revised)
    
    
 

In [ ]:
# how many tracks are in spotify, but don't have data?
    # use API to get audio_features?




In [ ]:
# check the percentage of songs accounted for




In [ ]:
# now merge with entire billboard list



# re-count the percentage of songs accounted for in the entire set

### 8+ M. Spotify Tracks, Genre, Audio Features (SQLite)
https://www.kaggle.com/datasets/maltegrosse/8-m-spotify-tracks-genre-audio-features

In [ ]:
url_8M_sql = 'D:\RYERSON\820\Datasets\8+ M. Spotify Tracks, Genre, Audio Features\spotify.sqlite'
url_8M_csv = 'all_audio_features_sql.csv'  # .gitignore (very big)


### Merge all Datasets into Billboard Features, 
export Billboard Features and All Features